# Encoding the Molecular Hamiltonian 

This notebook shows how to use a Fermion-Qubit encoding to encode a second quantised Molecular hamiltonian.

$$H = \sum_{i,j} h_{ij}a^{\dagger}_i a_j + \sum_{i,j,k,l} h_{ijkl}a^{\dagger}_i a^{\dagger}_j a_k a_l $$

## Simple Useage

Let's first get the coefficients for our second quantised hamiltonian.

For the time being we can use randomly generated ones.

In [1]:
import numpy as np
constant_energy = 0
one_e_coeffs = np.random.random((3,3))
two_e_coeffs = np.random.random((3,3,3,3))


Rather than figure out how many modes we need for an encoding, we can create one of the right size by passing in our coefficients.

In [2]:
from ferrmion import TernaryTree
tt = TernaryTree.from_hamiltonian_coefficients((one_e_coeffs, two_e_coeffs))

To encode a hamiltonian, we need both a mapping from fermonic operators to qubits, and an enumeration scheme which accounts for the degrees of freedom when labeling modes and qubits.

For the mapping, we'll use JordanWigner and for the enumeration scheme, we can use the default one.

Note we can [optimise our encoded hamiltonian](https://ferrmion.readthedocs.io/en/latest/notebooks/pauli_weight.html) by getting more clever with these.

In [3]:
jw = tt.JW()
jw.enumeration_scheme = jw.default_enumeration_scheme()
jw.enumeration_scheme

{'': (0, 0), 'z': (1, 1), 'zz': (2, 2)}

In [4]:
from ferrmion.hamiltonians import molecular_hamiltonian
hamiltonian = molecular_hamiltonian(encoding=jw, one_e_coeffs=one_e_coeffs, two_e_coeffs=two_e_coeffs, constant_energy=constant_energy)
hamiltonian

{'IXY': -0.17574123077898574j,
 'ZYY': (0.11998257282352547+0j),
 'ZXX': (0.11998257282352547+0j),
 'XIY': 0.01732924307278652j,
 'ZII': (-0.7088954319553701+0j),
 'ZYX': -0.0504625394348448j,
 'IYY': (0.24512284769699744+0j),
 'YZY': (0.35680829065388575+0j),
 'XXI': (0.18724188949278756+0j),
 'XYZ': 0.1300931107873033j,
 'YXI': 0.11709085128967568j,
 'IYX': 0.17574123077898574j,
 'IZZ': (-0.25075935245210035+0j),
 'XZX': (0.35680829065388575+0j),
 'YIY': (-0.056352858028815914+0j),
 'XYI': -0.11709085128967568j,
 'IZI': (-0.3208573192413564+0j),
 'XIX': (-0.05635285802881593+0j),
 'XZY': -0.21123275969333088j,
 'YYI': (0.18724188949278758+0j),
 'YZX': 0.2112327596933309j,
 'YXZ': -0.1300931107873033j,
 'IXX': (0.2451228476969975+0j),
 'YIX': -0.017329243072786535j,
 'YYZ': (-0.07582850656991245+0j),
 'IIZ': (0.032154087358560235+0j),
 'ZIZ': (0.09242364068555114+0j),
 'ZZI': (0.1515184677427007+0j),
 'XXZ': (-0.07582850656991247+0j),
 'III': (1.0044159078620143+0j),
 'ZXY': 0.0504625

## Hamiltonian Templates

Sometimes it can be useful to find out how an encoding behaves without providing coefficients. This lets us see which terms of the second quantised hamiltonian contribute to which pauli terms of the qubit Hamiltonian.

### Creating a Template

In [5]:
from ferrmion.hamiltonians import molecular_hamiltonian_template

The only information we need to provide is a set of XZ-encoded vectors (this is how ferrmion manipulates encodings internally) and imaginary factors for these vectors. 

In [6]:
ipowers, symplectics = jw._build_symplectic_matrix()

In [7]:
ipowers

array([0, 1, 0, 1, 0, 1], dtype=uint8)

In [8]:
np.array(symplectics, dtype=int)

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0],
       [0, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 1, 1]])

In [9]:
template = molecular_hamiltonian_template(ipowers, symplectics, physicist_notation=True)
template

{'XIX': {(1, 0, 2, 1): (-0.125+0j),
  (0, 1, 2, 1): (0.125+0j),
  (2, 1, 1, 0): (-0.125+0j),
  (1, 1, 0, 2): 0j,
  (2, 0, 1, 1): 0j,
  (1, 1, 2, 0): 0j,
  (0, 1, 1, 2): (-0.125+0j),
  (0, 2, 1, 1): 0j,
  (1, 0, 1, 2): (0.125+0j),
  (1, 2, 0, 1): (-0.125+0j),
  (2, 1, 0, 1): (0.125+0j),
  (1, 2, 1, 0): (0.125+0j)},
 'XZX': {(2, 1, 1, 0): (0.125+0j),
  (0, 1, 2, 1): (-0.125+0j),
  (0, 1, 1, 2): (0.125+0j),
  (2, 1, 0, 1): (-0.125+0j),
  (0, 2, 2, 2): 0j,
  (0, 2, 0, 0): 0j,
  (1, 2, 1, 0): (-0.125+0j),
  (1, 1, 2, 0): 0j,
  (0, 0, 0, 2): 0j,
  (0, 0, 2, 0): 0j,
  (2, 0, 0, 0): 0j,
  (1, 2, 0, 1): (0.125+0j),
  (2, 0, 2, 2): 0j,
  (0, 2): (0.25+0j),
  (2, 0, 1, 1): 0j,
  (1, 1, 0, 2): 0j,
  (2, 2, 0, 2): 0j,
  (1, 0, 2, 1): (0.125+0j),
  (2, 0): (0.25+0j),
  (2, 2, 2, 0): 0j,
  (0, 2, 1, 1): 0j,
  (1, 0, 1, 2): (-0.125+0j)},
 'YXZ': {(2, 2, 0, 1): 0j,
  (0, 1, 2, 2): 0j,
  (0, 2, 1, 2): -0.125j,
  (0, 2, 2, 1): 0.125j,
  (1, 0, 2, 2): 0j,
  (1, 2, 2, 0): -0.125j,
  (2, 2, 1, 0): 0j,
  (2,

### Filling a Template

So we can now see which modes contribute to each Pauli operator.

We can now fill out our template with the coefficients we used earlier.

This time we only need to provide a mapping from fermionic modes to majorana operators. Again we can use the default.

In [10]:
jw.default_mode_op_map

array([0, 1, 2], dtype=uint64)

In [11]:
from ferrmion.hamiltonians import fill_template
filled_template = fill_template(template,constant_energy=constant_energy, one_e_coeffs=one_e_coeffs, two_e_coeffs=two_e_coeffs, mode_op_map=jw.default_mode_op_map)
filled_template

{'XZY': -0.21123275969333088j,
 'ZYX': -0.050462539434844786j,
 'YZX': 0.21123275969333088j,
 'IIZ': (0.03215408735856026+0j),
 'ZZI': (0.1515184677427007+0j),
 'YIY': (-0.05635285802881593+0j),
 'XIX': (-0.056352858028815955+0j),
 'XYI': -0.11709085128967568j,
 'XYZ': 0.13009311078730323j,
 'IXY': -0.17574123077898576j,
 'IYX': 0.1757412307789858j,
 'XZX': (0.35680829065388575+0j),
 'ZYY': (0.11998257282352547+0j),
 'ZIZ': (0.09242364068555114+0j),
 'ZXX': (0.11998257282352547+0j),
 'YXZ': -0.1300931107873033j,
 'YZY': (0.35680829065388575+0j),
 'IZZ': (-0.25075935245210035+0j),
 'IXX': (0.2451228476969975+0j),
 'ZII': (-0.7088954319553701+0j),
 'IYY': (0.24512284769699744+0j),
 'ZXY': 0.050462539434844814j,
 'III': (1.0044159078620147+0j),
 'YXI': 0.11709085128967568j,
 'XXZ': (-0.07582850656991245+0j),
 'YYI': (0.18724188949278756+0j),
 'XXI': (0.18724188949278758+0j),
 'XIY': 0.01732924307278655j,
 'YYZ': (-0.07582850656991245+0j),
 'YIX': -0.017329243072786535j,
 'IZI': (-0.320857

Let' check to see that we have the same hamiltonian in each.

In [12]:
assert hamiltonian.keys() == filled_template.keys()
for k in hamiltonian.keys():
    # There is a little numerical instability so 
    # some values are different by 1e-18 or so!
    assert np.isclose(hamiltonian[k], filled_template[k])